In [6]:
import pandas as pd

# Loads the dataset
try:
    # Attempts to load the CSV file into a DataFrame.
    # It's a good practice to use a try-except block to handle
    # potential errors if the file is not found.
    df = pd.read_csv("winners_f1_1950_2025_v2.csv")
except FileNotFoundError:
    # If the file is not found, displays a clear error message
    # and exits the script, so the user knows what needs to be done.
    print("Error: The file 'winners_f1_1950_2025_v2.csv' was not found.")
    print("Make sure the file is in the same folder as this script.")
    exit()

# Data preprocessing and cleaning
# This is a crucial step to prepare the data for analysis.
# We convert the columns to the correct data types.
df['date'] = pd.to_datetime(df['date'])  # Converts the 'date' column to the datetime type, which allows for date-based operations.
df['time'] = pd.to_timedelta(df['time'], errors='coerce') # Converts the 'time' column to the timedelta type, which represents time durations. 'errors='coerce'' ensures invalid values become NaT (Not a Time), which will be handled in the next line.
df['time'] = df['time'].fillna(pd.Timedelta(seconds=0)) # Fills NaT values with zero. This prevents errors in later calculations.
df['time_segundos'] = df['time'].dt.total_seconds() # Creates a new column 'time_segundos' that stores the total race time in seconds, making mathematical calculations easier.

# Define the top 10 winning drivers
# For future analyses, we will focus only on the 10 drivers with the most victories in history.
top_10_pilotos = df['winner_name'].value_counts().head(10).index.tolist()
# 'value_counts()' counts the frequency of each driver.
# '.head(10)' selects the 10 most frequent.
# '.index.tolist()' extracts the names of these drivers and stores them in a list.

# Creates a filtered DataFrame containing only the wins of the top 10
# We use .copy() to ensure that any modifications to this new DataFrame do not affect the original.
df_top_10 = df[df['winner_name'].isin(top_10_pilotos)].copy()

# Dictionary to store points for each category
# This DataFrame will serve as our general "scoreboard," where we will sum the points for each driver.
pontos_por_categoria = pd.DataFrame({'winner_name': top_10_pilotos})

def atribuir_pontos(df_ranking, column_to_rank):
    """
    Helper function to rank and assign points (10 to 1).
    It is reused in all analyses to simplify the process.
    """
    # Ranks the drivers based on the scoring column, handling ties.
    df_ranking['rank'] = df_ranking[column_to_rank].rank(method='dense', ascending=False).astype(int)
    # Ensures the score is 11 - rank, for a descending score (10, 9, ...).
    # This transforms the rank (e.g., 1st, 2nd) into points (10, 9). If there is a tie for 1st place, both get 10 points.
    df_ranking['points'] = df_ranking['rank'].apply(lambda r: 11 - r)
    return df_ranking[['winner_name', 'points']]

# --- Analysis 1: Versatility in Teams and Circuits ---
# This analysis evaluates a driver's ability to win with different teams and at various circuits.
results_list = []
for driver in top_10_pilotos:
    driver_df = df_top_10[df_top_10['winner_name'] == driver]
    # Counts the number of teams with at least 5 wins.
    team_counts = driver_df['team'].value_counts()
    unique_teams_5_wins = len(team_counts[team_counts >= 5])
    # Counts the number of circuits with at least 3 wins.
    circuit_counts = driver_df['circuit'].value_counts()
    unique_circuits_3_wins = len(circuit_counts[circuit_counts >= 3])
    # Appends the results to the list.
    results_list.append({
        'winner_name': driver,
        'versatility_score': unique_teams_5_wins + unique_circuits_3_wins
    })
# Creates a DataFrame from the results and calculates the score.
versatility_df = pd.DataFrame(results_list).sort_values(by='versatility_score', ascending=False)
versatility_points = atribuir_pontos(versatility_df, 'versatility_score')
pontos_por_categoria = pd.merge(pontos_por_categoria, versatility_points, on='winner_name', how='left').rename(columns={'points': 'pontos_versatilidade'})

# --- Analysis 2: Important Wins Analysis ---
# This section focuses on victories at historically significant circuits.
important_circuits = ['Circuit de Spa Francorchamps', 'Circuit de Monaco', 'Autodromo Nazionale di Monza', 'Autodromo José Carlos Pace', 'Silverstone Circuit']
# Filters the DataFrame to include only wins at the important circuits and by the top 10 drivers.
important_wins_count = df[df['circuit'].isin(important_circuits) & df['winner_name'].isin(top_10_pilotos)].groupby('winner_name').size().reset_index(name='important_wins')
# Creates a DataFrame to ensure all 10 drivers are in the analysis, even if they have no important wins.
important_wins_ranking = pd.merge(pd.DataFrame({'winner_name': top_10_pilotos}), important_wins_count, on='winner_name', how='left').fillna(0)
important_wins_points = atribuir_pontos(important_wins_ranking, 'important_wins')
pontos_por_categoria = pd.merge(pontos_por_categoria, important_wins_points, on='winner_name', how='left').rename(columns={'points': 'pontos_momentos_importantes'})

# --- Analysis 3: Top Winner on Circuits ---
# This analysis identifies on how many circuits each driver is the top winner.
# Counts the number of wins for each driver on each circuit.
wins_per_circuit = df.groupby(['circuit', 'winner_name']).size().reset_index(name='wins')
# Finds the maximum number of wins for each circuit.
max_wins_per_circuit = wins_per_circuit.groupby('circuit')['wins'].max().reset_index(name='max_wins')
# Joins the tables to identify the drivers with the maximum number of wins per circuit.
top_winners_df = pd.merge(wins_per_circuit, max_wins_per_circuit, on='circuit').query('wins == max_wins')
# Counts on how many circuits each driver is a top winner.
circuits_top_winner_count = top_winners_df.groupby('winner_name').size().reset_index(name='top_winner_circuits')
# Creates a DataFrame to ensure all 10 drivers are in the analysis.
circuit_dominance_ranking = pd.merge(pd.DataFrame({'winner_name': top_10_pilotos}), circuits_top_winner_count, on='winner_name', how='left').fillna(0)
# Assigns points.
circuit_dominance_points = atribuir_pontos(circuit_dominance_ranking, 'top_winner_circuits')
pontos_por_categoria = pd.merge(pontos_por_categoria, circuit_dominance_points, on='winner_name', how='left').rename(columns={'points': 'pontos_dominio'})

# --- Analysis 4: Longevity (8+ Wins per Year) ---
# This analysis measures a driver's longevity based on years of high performance.
# Counts the number of wins for each driver per year.
wins_per_year = df.groupby(['winner_name', 'year']).size().reset_index(name='wins')
# Filters for years where the driver had at least 8 wins.
longevity_years = wins_per_year[wins_per_year['wins'] >= 8]
# Calculates the longevity period.
longevity_period = longevity_years.groupby('winner_name')['year'].agg(min_year='min', max_year='max').reset_index()
longevity_period['longevity_period_years'] = longevity_period['max_year'] - longevity_period['min_year'] + 1
# Merges the results with the list of the top 10, filling with 0.
longevity_ranking = pd.merge(pd.DataFrame({'winner_name': top_10_pilotos}), longevity_period[['winner_name', 'longevity_period_years']], on='winner_name', how='left').fillna(0)
# Assigns the score, handling ties.
longevity_points = atribuir_pontos(longevity_ranking, 'longevity_period_years')
pontos_por_categoria = pd.merge(pontos_por_categoria, longevity_points, on='winner_name', how='left').rename(columns={'points': 'pontos_longevidade'})

# --- Analysis 5: Efficiency in Wins ---
# This analysis evaluates a driver's "efficiency" in winning, based on average lap time.
# Calculates the average time per lap for each victory.
df_top_10['avg_time_per_lap'] = df_top_10['time_segundos'] / df_top_10['laps']
# Calculates the overall average time per lap for each driver.
efficiency_ranking = df_top_10.groupby('winner_name')['avg_time_per_lap'].mean().reset_index(name='avg_time_per_lap_s')
# Assigns the score, handling ties.
efficiency_ranking['rank'] = efficiency_ranking['avg_time_per_lap_s'].rank(method='dense', ascending=True).astype(int)
efficiency_ranking['points'] = efficiency_ranking['rank'].apply(lambda r: 11 - r)
efficiency_points = efficiency_ranking[['winner_name', 'points']]
pontos_por_categoria = pd.merge(pontos_por_categoria, efficiency_points, on='winner_name', how='left').rename(columns={'points': 'pontos_eficiencia'})

# --- Analysis 6: Longest Winning Streak ---
# This analysis identifies the longest consecutive winning streak for each driver.
# Sorts the complete DataFrame by date to identify sequences.
df_sorted = df.sort_values(by='date').reset_index(drop=True)
# Identifies the start of a new streak.
df_sorted['is_new_streak'] = (df_sorted['winner_name'] != df_sorted['winner_name'].shift(1)).cumsum()
# Counts the length of each winning streak.
winning_streaks = df_sorted.groupby(['winner_name', 'is_new_streak']).size().reset_index(name='streak_length')
# Finds the longest streak for each driver.
longest_streaks = winning_streaks.groupby('winner_name')['streak_length'].max().reset_index(name='streak_length')
# Filters the streak analysis for the top 10 drivers.
streak_ranking = pd.merge(pd.DataFrame({'winner_name': top_10_pilotos}), longest_streaks, on='winner_name', how='left').fillna(0)
# Assigns the score, handling ties.
streak_points = atribuir_pontos(streak_ranking, 'streak_length')
pontos_por_categoria = pd.merge(pontos_por_categoria, streak_points, on='winner_name', how='left').rename(columns={'points': 'pontos_sequencia_vitorias'})

# --- Final Score Calculation and Ranking ---
# This section consolidates the points from all six analyses to create a general ranking.
# Sums the points from all categories to get the total score.
pontos_por_categoria['Total Score'] = pontos_por_categoria.iloc[:, 1:].sum(axis=1)
# Sorts the final DataFrame based on the total score.
ranking_final = pontos_por_categoria.sort_values(by='Total Score', ascending=False).reset_index(drop=True)
# Assigns the final rank.
ranking_final['Rank'] = ranking_final['Total Score'].rank(method='min', ascending=False).astype(int)

# Displays the final ranking.
print("\nTop 10 Best Drivers")
print(ranking_final[['Rank', 'winner_name', 'Total Score']].to_markdown(index=False, numalign="left", stralign="left"))


Top 10 Best Drivers
| Rank   | winner_name        | Total Score   |
|:-------|:-------------------|:--------------|
| 1      | Michael Schumacher | 55            |
| 2      | Lewis Hamilton     | 48            |
| 2      | Max Verstappen     | 48            |
| 4      | Alain Prost        | 41            |
| 5      | Sebastian Vettel   | 40            |
| 6      | Ayrton Senna       | 38            |
| 7      | Nigel Mansell      | 34            |
| 8      | Jim Clark          | 29            |
| 9      | Fernando Alonso    | 28            |
| 10     | Jackie Stewart     | 26            |


In [5]:
# It's important to note that this is an analysis using only one dataset, ignoring various factors and concepts that influence the final result
# (modernization of cars, circuits that no longer exist, new circuits, etc.). 
# This project is simply an analysis of the dataset we have access to, with a focus on using pandas for exploratory analysis with a specific goal.